# Simple example showcasing some of the features of STUBS

Geometry is divided into 4 domains; two volumes, and two surfaces:
- PM
- Cytosol
- Cytosol

TODO: Add some proper definitions of the geometries

There are three function-spaces on the three domains:
```
- u[Cyto] = [A, B]
- u[ERm]  = [R, Ro]
- u[ER]   = [AER]
```

TODO: Write out the equations

Roughly, this model is similar to an IP3 pulse at the PM, leading to Ca2+ release at the ER

In [ ]:
import os
import logging

import dolfin as d
import sympy as sym

from smart import config, common, mesh, model, mesh_tools
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from smart.units import unit

We will set the logging level to `INFO`. This will display some output during the simulation. If you want to get even more output you could set the logging level to `DEBUG`.

In [ ]:
logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

Futhermore, you could also save the logs to a file by attaching a file handler to the logger as follows.

```python
file_handler = logging.FileHandler("filename.log")
file_handler.setFormatter(logging.Formatter(smart.config.base_format))
logger.addHandler(file_handler)
```

First, we define the various units for the inputs

In [ ]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM
surf_unit = molecule / um**2

## Generate model
Next we generate the model.


### Species
TODO: Write about the species. What is a the purpose of this, and how does this fit into the model
name, initial concentration, concentration units, diffusion, diffusion units, compartment

In [ ]:
A = Species("A", 0.01, vol_unit, 1.0, D_unit, "Cyto")
B = Species("B", 0.0, vol_unit, 1.0, D_unit, "Cyto")
AER = Species("AER", 200.0, vol_unit, 5.0, D_unit, "ER")
# Lets create an algebraic expression to define the initial condition of R
Rinit = "(sin(40*y) + cos(40*z) + sin(40*x) + 3) * (y-x)**2"
R1 = Species("R1", Rinit, surf_unit, 0.02, D_unit, "ERm")
R1o = Species("R1o", 0.0, surf_unit, 0.02, D_unit, "ERm")
# R2    = Species('R2'  , Rinit, surf_unit, 0   , D_unit, 'ERm')

Create a species container

In [ ]:
sc = SpeciesContainer()
sc.add([R1o, R1, AER, B, A])

### Compartments

TODO: Write about the compartments. What is a the purpose of this, and how does this fit into the model

In [ ]:
Cyto = Compartment("Cyto", 3, um, 1)
PM = Compartment("PM", 2, um, 10)
ER = Compartment("ER", 3, um, 2)
ERm = Compartment("ERm", 2, um, 12)
PM.specify_nonadjacency(['ERm', 'ER'])
ERm.specify_nonadjacency(['PM'])

Create a compartment container

In [ ]:
cc = CompartmentContainer()
cc.add([ERm, ER, PM, Cyto])

###  Parameters and Reactions

TODO: Add more info
Pulse function for B input at the PM
One way to prescribe a "pulse-like" flux is to define the flux as the derivative of a sigmoid
(here we choose atan as the sigmoid because of its simple derivative)

In [ ]:
Vmax, t0, m = 500, 0.1, 200
t = sym.symbols("t")
pulseI = Vmax * sym.atan(m * (t - t0))
pulse = sym.diff(pulseI, t)
j1pulse = Parameter.from_expression(
    "j1pulse", pulse, flux_unit, use_preintegration=True, preint_sym_expr=pulseI
)
r1 = Reaction(
    "r1",
    [],
    ["B"],
    param_map={"J": "j1pulse"},
    eqn_f_str="J",
    explicit_restriction_to_domain="PM",
)

# Degradation of B in the cytosol
k2f = Parameter("k2f", 10, 1 / sec)
r2 = Reaction(
    "r2", ["B"], [], param_map={"on": "k2f"}, reaction_type="mass_action_forward"
)

# Activating receptors on ERm with B
k3f = Parameter("k3f", 100, 1 / (uM * sec))
k3r = Parameter("k3r", 100, 1 / sec)
r3 = Reaction("r3", ["B", "R1"], ["R1o"], {"on": "k3f", "off": "k3r"})
Reaction()
# Release of A from ERm to cytosol
k4Vmax = Parameter("k4Vmax", 2000, 1 / (uM * sec))
r4 = Reaction(
    "r4",
    ["AER"],
    ["A"],
    param_map={"Vmax": "k4Vmax"},
    species_map={"R1o": "R1o", "uER": "AER", "u": "A"},
    eqn_f_str="Vmax*R1o*(uER-u)",
)

Create containers

In [ ]:
pc = ParameterContainer()
pc.add([k4Vmax, k3r, k3f, k2f, j1pulse])

rc = ReactionContainer()
rc.add([r4, r3, r2, r1])

We load the model generated above, and load in the mesh we will use in this example.


## Create/load in mesh

In SMART we have different levels of meshes. Here we create a UnitCube mesh defined by

$$
\Omega = [0, 1] \times [0, 1] \times [0, 1] \subset \mathbb{R}^3
$$

which will serve as out parent mesh

There is also the notion of child meshes....

In [ ]:
domain, facet_markers, cell_markers = common.DemoCuboidsMesh()

Since ? we turn of PM? on all sides. We do this by setting the facet markers whose $x$ value is greater than zero and that are already marked as ? (How do we know that the value we select should be 10?)

In [ ]:

# Base mesh
domain, facet_markers, cell_markers = mesh_tools.DemoCuboidsMesh()
# Turn off "PM" on all sides of the cube except x=0
for face in d.faces(domain):
    if face.midpoint().x() > d.DOLFIN_EPS and facet_markers[face] == 10:
        facet_markers[face] = 0


In order to load this into a `ParentMesh` object, we need to first save it

In [ ]:
os.makedirs("mesh", exist_ok=True)
mesh_tools.write_mesh(domain, facet_markers, cell_markers, filename="mesh/DemoCuboidsMesh")

Now we can create a parent mesh. Explain what this is. Can we visualize it?

In [ ]:
parent_mesh = mesh.ParentMesh(
    mesh_filename="mesh/DemoCuboidsMesh.h5",
    mesh_filetype="hdf5",
    name="parent_mesh",
)

Now we are ready to set up the model. First we load the default configurations.



In [ ]:
conf = config.Config()
conf.solver.update(
    {
        "final_t": 1,
        "initial_dt": 0.01,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)
print(conf)


We create a model using the different containers and the parent mesh. 

In [ ]:
model = model.Model(pc, sc, cc, rc, conf, parent_mesh)
print(model)

Next we need to initialize the model

In [ ]:
model.initialize(initialize_solver=False)
model.initialize_discrete_variational_problem_and_solver()

We create some XDMF files where we will store the output 

In [ ]:

# Write initial condition(s) to file
results = dict()
os.makedirs("results", exist_ok=True)
for species_name, species in model.sc.items:
    results[species_name] = d.XDMFFile(
        model.mpi_comm_world, f"results/{species_name}.xdmf"
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(model.sc[species_name].u["u"], model.t)

We now run the the solver

In [ ]:
# Solve
while model.t <  model.final_t:
    # Solve the system
    model.monolithic_solve()
    # Save results for post processing
    for species_name, species in model.sc.items:
        results[species_name].write(model.sc[species_name].u["u"], model.t)


Could we visualize the solution?